In [4]:
using Flux
using BSON: @load
using Revise
using FFTW: plan_dct
using GenerativeRecoveries
using GenerativeRecoveries: get_recovery_errors_tocompare_frequencysamplingalgorithms, wrap_model_withreshape, plot_recovery_errors_tocompare_frequencysamplingalgorithms, _setupMNISTimagesignals, samplefrequenciesuniformly, samplefrequenciesuniformlyanddeterministically, _setupmodel, IndexedMatrix, recoversignal, runexperimenttensor, _setupfrequencies, _plot_tables_ofrecoveries, plot_MNISTrecoveries, plot_MNISTrecoveryerrors
@load "../savedmodels/more_incoherentepoch20" model
model = wrap_model_withreshape(model)
using DataFrames
using DataFrames: allcombinations, DataFrame, transform!
using LinearAlgebra: norm
using Statistics
using AlgebraOfGraphics, CairoMakie
using CairoMakie

In [13]:
presigmoid = true
inrange = true
datasplit = :test
kwargs = (;)
img_size = (28, 28)
images = [0,8,9]
aimed_ms = [8, 16, 24,32,50,64,100,128,256, 784]
aimed_ms = [8,10,16,24,32,64, 128,784]
set_aog_theme!()

In [14]:
model = _setupmodel(model; presigmoid)
decoder = model.decoder
truesignals = _setupMNISTimagesignals(images, model; presigmoid, inrange, datasplit, kwargs...)

3-element Vector{Matrix{Float32}}:
 [-7.469619 -8.210722 … -6.887391 -6.9714537; -8.185194 -8.266231 … -6.0869474 -6.5387964; … ; -7.3637094 -7.4615426 … -7.2899294 -6.6202765; -8.471201 -8.386674 … -6.8756485 -7.1914954]
 [-12.102797 -10.683401 … -10.619436 -9.775031; -9.532698 -8.710725 … -10.3775425 -11.252632; … ; -10.273352 -8.648103 … -11.889959 -11.2867775; -10.695029 -9.536697 … -10.058863 -11.508093]
 [-7.3899217 -8.357929 … -6.8418264 -7.179199; -7.94391 -7.7998304 … -8.036548 -9.812468; … ; -8.386164 -6.5711365 … -8.841705 -8.660866; -6.769309 -7.662223 … -8.889787 -10.571924]

In [17]:

# not well adapted to the in range case with many models; for that we need a notion of dependence when running experiments. Probably a callback.
pdct = plan_dct(truesignals[1])
recoveryalgs = [:recoversignal, :relaxed_recover]
recoverydict = Dict(:recoversignal => recoversignal, :relaxed_recover => relaxed_recover)
frame = allcombinations(DataFrame, (:truesignal => truesignals), (:numfrequencies => aimed_ms), (:recoveryalg => recoveryalgs))
transform!(frame, [:numfrequencies] => ByRow((m) -> samplefrequenciesuniformly(m, img_size)) => :sampledfrequencies)


Row,truesignal,numfrequencies,recoveryalg,sampledfrequencies
,Array…,Int64,Symbol,Array…
1,Float32[-7.46962 -8.21072 … -6.88739 -6.97145; -8.18519 -8.26623 … -6.08695 -6.5388; … ; -7.36371 -7.46154 … -7.28993 -6.62028; -8.4712 -8.38667 … -6.87565 -7.1915],8,recoversignal,Bool[0 0 … 0 0; 0 0 … 1 0; … ; 0 0 … 0 0; 1 0 … 0 0]
2,Float32[-12.1028 -10.6834 … -10.6194 -9.77503; -9.5327 -8.71072 … -10.3775 -11.2526; … ; -10.2734 -8.6481 … -11.89 -11.2868; -10.695 -9.5367 … -10.0589 -11.5081],8,recoversignal,Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]
3,Float32[-7.38992 -8.35793 … -6.84183 -7.1792; -7.94391 -7.79983 … -8.03655 -9.81247; … ; -8.38616 -6.57114 … -8.84171 -8.66087; -6.76931 -7.66222 … -8.88979 -10.5719],8,recoversignal,Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]
4,Float32[-7.46962 -8.21072 … -6.88739 -6.97145; -8.18519 -8.26623 … -6.08695 -6.5388; … ; -7.36371 -7.46154 … -7.28993 -6.62028; -8.4712 -8.38667 … -6.87565 -7.1915],10,recoversignal,Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]
5,Float32[-12.1028 -10.6834 … -10.6194 -9.77503; -9.5327 -8.71072 … -10.3775 -11.2526; … ; -10.2734 -8.6481 … -11.89 -11.2868; -10.695 -9.5367 … -10.0589 -11.5081],10,recoversignal,Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]
6,Float32[-7.38992 -8.35793 … -6.84183 -7.1792; -7.94391 -7.79983 … -8.03655 -9.81247; … ; -8.38616 -6.57114 … -8.84171 -8.66087; -6.76931 -7.66222 … -8.88979 -10.5719],10,recoversignal,Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]
7,Float32[-7.46962 -8.21072 … -6.88739 -6.97145; -8.18519 -8.26623 … -6.08695 -6.5388; … ; -7.36371 -7.46154 … -7.28993 -6.62028; -8.4712 -8.38667 … -6.87565 -7.1915],16,recoversignal,Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]
8,Float32[-12.1028 -10.6834 … -10.6194 -9.77503; -9.5327 -8.71072 … -10.3775 -11.2526; … ; -10.2734 -8.6481 … -11.89 -11.2868; -10.695 -9.5367 … -10.0589 -11.5081],16,recoversignal,Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]
9,Float32[-7.38992 -8.35793 … -6.84183 -7.1792; -7.94391 -7.79983 … -8.03655 -9.81247; … ; -8.38616 -6.57114 … -8.84171 -8.66087; -6.76931 -7.66222 … -8.88979 -10.5719],16,recoversignal,Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]


In [21]:

function experimentgetrelative_error(truesignal, frequency, model, pdct, recoveryfn; multithread=true, kwargs...)
    #if multithread
    #     pdct = deepcopy(pdct)
    # end
    pdct = deepcopy(pdct)
    A = IndexedMatrix(pdct, frequency)
    measurements = A * truesignal
    recoveryimg = recoveryfn(measurements, A, model; kwargs...)
    relativeerr = norm(recoveryimg .- truesignal) / norm(truesignal)
    Dict(:relerr => relativeerr)
end

experimentgetrelative_error (generic function with 1 method)

In [25]:
kwargs = (;max_iter = 10)


(max_iter = 10,)

In [26]:

transform!(frame, [:truesignal, :sampledfrequencies, :recoveryalg] => ByRow((truesignal, frequency, recoveryalg) -> experimentgetrelative_error(truesignal, frequency, decoder, pdct, recoverydict[recoveryalg]; kwargs...)) => AsTable)

┌ Warning: Max num. iterations reached
└ @ GenerativeRecoveries /Users/matthewscott/Prog/GenerativeRecoveries/src/recoveryalgorithms.jl:20
┌ Warning: Max num. iterations reached
└ @ GenerativeRecoveries /Users/matthewscott/Prog/GenerativeRecoveries/src/recoveryalgorithms.jl:20


┌ Warning: Max num. iterations reached
└ @ GenerativeRecoveries /Users/matthewscott/Prog/GenerativeRecoveries/src/recoveryalgorithms.jl:20
┌ Warning: Max num. iterations reached
└ @ GenerativeRecoveries /Users/matthewscott/Prog/GenerativeRecoveries/src/recoveryalgorithms.jl:20
┌ Warning: Max num. iterations reached
└ @ GenerativeRecoveries /Users/matthewscott/Prog/GenerativeRecoveries/src/recoveryalgorithms.jl:20
┌ Warning: Max num. iterations reached
└ @ GenerativeRecoveries /Users/matthewscott/Prog/GenerativeRecoveries/src/recoveryalgorithms.jl:20
┌ Warning: Max num. iterations reached
└ @ GenerativeRecoveries /Users/matthewscott/Prog/GenerativeRecoveries/src/recoveryalgorithms.jl:20
┌ Warning: Max num. iterations reached
└ @ GenerativeRecoveries /Users/matthewscott/Prog/GenerativeRecoveries/src/recoveryalgorithms.jl:20
┌ Warning: Max num. iterations reached
└ @ GenerativeRecoveries /Users/matthewscott/Prog/GenerativeRecoveries/src/recoveryalgorithms.jl:20
┌ Warning: Max num. iterati

┌ Warning: Max num. iterations reached
└ @ GenerativeRecoveries /Users/matthewscott/Prog/GenerativeRecoveries/src/recoveryalgorithms.jl:20
┌ Warning: Max num. iterations reached
└ @ GenerativeRecoveries /Users/matthewscott/Prog/GenerativeRecoveries/src/recoveryalgorithms.jl:20
┌ Warning: Max num. iterations reached
└ @ GenerativeRecoveries /Users/matthewscott/Prog/GenerativeRecoveries/src/recoveryalgorithms.jl:20
┌ Warning: Max num. iterations reached
└ @ GenerativeRecoveries /Users/matthewscott/Prog/GenerativeRecoveries/src/recoveryalgorithms.jl:20
┌ Warning: Max num. iterations reached
└ @ GenerativeRecoveries /Users/matthewscott/Prog/GenerativeRecoveries/src/recoveryalgorithms.jl:20
┌ Warning: Max num. iterations reached
└ @ GenerativeRecoveries /Users/matthewscott/Prog/GenerativeRecoveries/src/recoveryalgorithms.jl:20
┌ Warning: Max num. iterations reached
└ @ GenerativeRecoveries /Users/matthewscott/Prog/GenerativeRecoveries/src/recoveryalgorithms.jl:20
┌ Warning: Max num. iterati

UndefVarError: UndefVarError: indexof not defined

In [ ]:
frame = combine(groupby(frame, [:numfrequencies, :recoveryalg]), :relerr, :algname, :relerr => mean => :meanrelerr, :relerr => std => :std_deviation, :relerr => (x -> 10^(mean(log10.(x)) - std(log10.(x)))) => :botuncert, :relerr => (x -> 10^(mean(log10.(x)) + std(log10.(x)))) => :topuncert)